## Need to consider ALL owners -- use a different generator dataset

In [17]:
import pandas as pd
import json
pd.options.display.max_rows = 999

In [18]:
dataDir = '/Users/rachelanderson/Dropbox (Princeton)/Data for Tax Equity Project/eia_860m/'
jsonDir = '/Users/rachelanderson/Dropbox (Princeton)/Data for Tax Equity Project/JSONFiles/'

In [25]:
with open(jsonDir + 'final_parent_dict.json') as outfile:
    parent_map = json.load(outfile)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/rachelanderson/Dropbox (Princeton)/Data for Tax Equity Project/JSONFiles/parent_utility_dict.json'

In [20]:
#import data
data = pd.ExcelFile(dataDir + 'february_generator2020.xlsx')
df = data.parse(sheetname=data.sheet_names[0], skiprows=1)

#clean column names
df.columns = [x.lower().replace(" ","_") for x in df.columns]
df.columns = [x.split('_(mw)')[:-1][0] if "(mw)" in x else x for x in df.columns]

In [21]:
df['company_name'] = df.entity_name.str.lower()
df['parent_company'] = df.company_name.map(parent_map)

In [22]:
top_owner_list = df[df['energy_source_code']=='SUN'].groupby('parent_company').nameplate_capacity.sum().sort_values(ascending=False).reset_index().head(20)

In [23]:
top_owner_list

,parent_company,nameplate_capacity
0,95.0,3780.1
1,128.0,2545.1
2,1248.0,2255.6
3,1442.0,2011.1
4,1348.0,1393.8
5,227.0,1215.3
6,10.0,1033.8
7,1465.0,868.0
8,566.0,827.3
9,320.0,740.9


In [24]:
df[(df['energy_source_code']=='SUN') & (df['operating_year']>2017)].groupby('parent_company').nameplate_capacity.sum().sort_values(ascending=False).reset_index().head(20)

,parent_company,nameplate_capacity
0,95.0,1969.7
1,1442.0,502.2
2,3522.0,495.1
3,566.0,402.3
4,1248.0,383.5
5,1348.0,366.7
6,3534.0,300.0
7,10.0,297.9
8,363.0,289.4
9,443.0,283.5


In [9]:
top_owner_list.nameplate_capacity.sum()

20147.4

In [10]:
df[df['energy_source_code']=='SUN'].nameplate_capacity.sum()

39250.8

## Fact 1: The top 20 utilities (lower bound estimate) own 50% of solar capacity. 

In [11]:
df[df['energy_source_code']=='SUN'].parent_company.nunique()

1268

In [12]:
solar =df[df['energy_source_code']=='SUN']

In [13]:
top_owner_list = top_owner_list.parent_company

In [14]:
solar[solar['parent_company'].isin(top_owner_list)]

,entity_id,entity_name,plant_id,plant_name,sector,plant_state,generator_id,unit_code,nameplate_capacity,net_summer_capacity,...,planned_uprate_month,planned_uprate_of_summer_capacity,county,latitude,longitude,google_map,bing_map,balancing_authority_code,company_name,parent_company
1109,18454,Tampa Electric Co,645.0,Big Bend,Electric Utility,FL,1,NaN,19.0,19.4,...,,,Hillsborough,27.794400,-82.40360,Map,Map,TEC,tampa electric co,emera
14473,56203,Solar Star NAFB LLC,56568.0,Nellis Air Force Base Solar Array,Commercial Non-CHP,NV,NS1,NaN,4.9,4.9,...,,,Clark,36.260278,-115.05440,Map,Map,NEVP,solar star nafb llc,solar star
14474,56203,Solar Star NAFB LLC,56568.0,Nellis Air Force Base Solar Array,Commercial Non-CHP,NV,NS2,NaN,5.2,5.2,...,,,Clark,36.260278,-115.05440,Map,Map,NEVP,solar star nafb llc,solar star
14475,56203,Solar Star NAFB LLC,56568.0,Nellis Air Force Base Solar Array,Commercial Non-CHP,NV,NS3,NaN,3.9,3.9,...,,,Clark,36.260278,-115.05440,Map,Map,NEVP,solar star nafb llc,solar star
15066,56023,Arizona Solar One LLC,56812.0,Solana Generating Station,IPP Non-CHP,AZ,GEN 1,NaN,140.0,147.8,...,,,Maricopa,32.922294,-112.95590,Map,Map,AZPS,arizona solar one llc,arizona solar one llc
15067,56023,Arizona Solar One LLC,56812.0,Solana Generating Station,IPP Non-CHP,AZ,GEN2,NaN,140.0,147.8,...,,,Maricopa,32.922294,-112.95590,Map,Map,AZPS,arizona solar one llc,arizona solar one llc
15195,57170,"EDF Renewable Asset Holdings, Inc.",56875.0,Sacramento Soleil LLC,IPP Non-CHP,CA,TBD,NaN,1.0,1,...,,,Sacramento,38.449790,-121.16470,Map,Map,BANC,"edf renewable asset holdings, inc.","edf renewable asset holdings, inc."
15248,57044,"Constellation Solar New Jersey, LLC",56900.0,Atlantic City Convention Center,Commercial Non-CHP,NJ,1,NaN,0.5,0.5,...,,,Atlantic,39.363900,-74.44080,Map,Map,PJM,"constellation solar new jersey, llc",exelon
15249,57044,"Constellation Solar New Jersey, LLC",56900.0,Atlantic City Convention Center,Commercial Non-CHP,NJ,2,NaN,0.5,0.5,...,,,Atlantic,39.363900,-74.44080,Map,Map,PJM,"constellation solar new jersey, llc",exelon
15250,57044,"Constellation Solar New Jersey, LLC",56900.0,Atlantic City Convention Center,Commercial Non-CHP,NJ,3,NaN,0.5,0.5,...,,,Atlantic,39.363900,-74.44080,Map,Map,PJM,"constellation solar new jersey, llc",exelon


In [105]:
solar.sort_values(by='nameplate_capacity',ascending=False)[['company_name','nameplate_capacity','parent_company']].head(20)

,company_name,nameplate_capacity,parent_company
16039,av solar ranch 1 llc,253.0,exelon
20639,"cd arevon usa, inc.",252.3,"cd arevon usa, inc."
21503,"phoebe energy project, llc",250.0,"phoebe energy project, llc"
16654,"cd arevon usa, inc.",250.0,"cd arevon usa, inc."
21803,misae lessee llc,240.0,NaN
19339,southern power co,205.3,southern company
19440,"buckthorn westex, llc",202.0,clearway energy
21523,techren solar ii llc,200.0,techren solar ii llc
18867,wright solar park llc,200.0,wright solar park llc
19622,southern power co,185.0,southern company
